In [5]:
import requests
import os
from dotenv import load_dotenv
from pprint import pprint
import json


# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}

# query string 문자열을 dict 선언
payload = {
    'query': '파이썬',
    'display': 10,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/book.json' #?query=파이썬&display=100&sort=sim

# requests get(url, params, headers) 요청 
res = requests.get(url, params=payload, headers=headers)
# json() 함수로 응답 결과 가져오기
items_data = res.json()['items']

pprint(items_data)

[{'author': '박응용',
  'description': '프로그래밍 분야 8년 연속 베스트셀러!\n'
                 '《Do it! 점프 투 파이썬》 전면 개정 2판 출시!\n'
                 '\n'
                 '중고등학생도, 비전공자도, 직장인도 프로그래밍에 눈뜨게 만든 바로 그 책이 전면 개정 2판으로 새롭게 '
                 '태어났다! 챗GPT를 시작으로 펼쳐진 생성 AI 시대에 맞춰 설명과 예제를 다듬고, 최신 경향과 심화 내용을 '
                 '보충했다. 또한 이번 개정 2판도 50만 코딩 유튜버인 조코딩과 협업을 통해 유튜브 동영상을 제공해 파이썬을 '
                 '더 쉽게 공부할 수 있다.\n'
                 '\n'
                 '8년 연속 베스트셀러! 위키독스 누적 방문 300만! 독자의 입에서 입으로 전해진 추천과 수많은 대학 및 '
                 '학원의 교재 채택을 통해 검증은 이미 끝났다. 코딩을 처음 배우는 중고등학생부터 코딩 소양을 기르려는 '
                 '비전공자, 자기계발에 진심인 직장인까지! 이 책과 함께 파이썬 프로그래밍의 세계로 ‘점프’해 보자!',
  'discount': '19800',
  'image': 'https://shopping-phinf.pstatic.net/main_4035408/40354085633.20230927071024.jpg',
  'isbn': '9791163034735',
  'link': 'https://search.shopping.naver.com/book/catalog/40354085633',
  'pubdate': '20230615',
  'publisher': '이지스퍼블리싱',
  'title': 'Do it! 점프 투 파이썬 (중학생도 첫날부터 실습하는 초고속 입문서)'},
 {'author': '

In [6]:
def search_books(query, display=50): 
    # query string 문자열을 dict 선언
    payload = {
        'query': query,
        'display': display,
        'sort': 'sim'
    }

    url = 'https://openapi.naver.com/v1/search/book.json' #?query=파이썬&display=100&sort=sim

    # requests get(url, params, headers) 요청 
    res = requests.get(url, params=payload, headers=headers)
    # json() 함수로 응답 결과 가져오기
    items_data = res.json()['items']

    return items_data

def search_shops(query, display=50): 
    # query string 문자열을 dict 선언
    payload = {
        'query': query,
        'display': display,
        'sort': 'sim'
    }

    url = 'https://openapi.naver.com/v1/search/shop.json' #?query=파이썬&display=100&sort=sim

    res = requests.get(url, params=payload, headers=headers)
    # json() 함수로 응답 결과 가져오기
    items_data = res.json()['items']
    return items_data

def save_json(items_data):
    with open('../data/books.json','w',encoding='utf-8') as file:
        json.dump(items_data, file)

def save_json_shop(items_data):
    with open('../data/shops.json','w',encoding='utf-8') as file:
        json.dump(items_data, file)


if __name__ == '__main__':
    save_json(search_books('파이썬'))        
    save_json_shop(search_shops('가디건'))


리팩토링 된 코드

In [7]:
import requests
import os
from dotenv import load_dotenv
import json

# .env 파일에서 환경 변수 로드
load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}


def search_naver_api(endpoint, query, display=50):
    """네이버 API 검색 함수"""
    payload = {
        'query': query,
        'display': display,
        'sort': 'sim'
    }
    url = f'https://openapi.naver.com/v1/search/{endpoint}.json'
    res = requests.get(url, params=payload, headers=headers)
    res.raise_for_status()  # 에러 발생 시 예외 처리
    return res.json().get('items', [])


def save_json(data, filepath):
    """JSON 파일 저장 함수"""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)


if __name__ == '__main__':
    books = search_naver_api('book', '파이썬')
    save_json(books, '../data/books.json')

    shops = search_naver_api('shop', '가디건')
    save_json(shops, '../data/shops.json')

In [9]:
import pandas as pd

books_df = pd.read_json('../data/books.json')
print(type(books_df))
books_df.head(2)
books_df.loc[books_df['discount'] >= 20000,['title','author','discount','publisher','pubdate']]\
    .sort_values(by='discount', ascending=False).reset_index(drop=True)

<class 'pandas.core.frame.DataFrame'>


,title,author,discount,publisher,pubdate
0,으뜸 파이썬 (프로그래밍을 사랑하는 두 교수가 작정하고 쓴),박동규,32000,생능출판,20200217
1,파이썬 통계학 기초 (단계별 접근),문창권^박수용^양갑규^정명수,31500,탐진출판사,20250811
2,단단한 파이썬 (더 깔끔하고 관리가 쉬운 파이썬 코드를 위해),패트릭 비아포어,31500,에이콘출판,20220824
3,파이썬,Y. Daniel Liang,31500,에피스테메,20180302
4,으뜸 파이썬 (개정판),박동규^강영민,31280,생능출판,20240614
5,파이썬,홍의경,29440,생능출판,20220309
6,파이썬 마스터 (실생활 융합 예제로 배우는),김종훈^김동건,29440,한빛아카데미,20250117
7,객체지향 파이썬 (파이게임과 GUI로 객체지향 프로그래밍 정복하기),어브 캘브,28800,제이펍,20231013
8,독학 파이썬,야마다 요시히로,28800,정보문화사,20241220
9,새내기 파이썬,천인국,27600,생능출판,20220630


In [11]:
# 함수로 정의
def filter_and_sort_books(df, min_discount=20000):
    """
    할인 금액이 min_discount 이상인 도서 필터링 후 정렬
    
    Parameters:
        df (DataFrame): 도서 데이터프레임
        min_discount (int): 최소 할인 금액 기준 (기본값 20000)

    Returns:
        DataFrame: 필터링 및 정렬된 결과
    """
    return (
        df.loc[df['discount'] >= min_discount, ['title', 'author', 'discount', 'publisher', 'pubdate']]
          .sort_values(by='discount', ascending=False)
          .reset_index(drop=True)
    )

filter_and_sort_books(books_df,10000)
books_df.columns

Index(['title', 'link', 'image', 'author', 'discount', 'publisher', 'pubdate',
       'isbn', 'description'],
      dtype='object')

In [12]:
print(type(books_df['publisher']))
print(type(books_df['publisher'].str))

<class 'pandas.core.series.Series'>
<class 'pandas.core.strings.accessor.StringMethods'>


In [13]:
books_df.columns.drop(['image','description'])

Index(['title', 'link', 'author', 'discount', 'publisher', 'pubdate', 'isbn'], dtype='object')

In [14]:
# image , description 컬럼은 제외한 모든 컬럼 출력하기
books_df.loc[books_df['publisher'].str.contains('인피니티북스'),\
    books_df.columns.drop(['image','description'])].reset_index(drop=True)

,title,link,author,discount,publisher,pubdate,isbn
0,파이썬,https://search.shopping.naver.com/book/catalog...,천인국,10800,인피니티북스,20170830,9791185578330
1,파이썬 플러스,https://search.shopping.naver.com/book/catalog...,최희식,22500,인피니티북스,20240731,9791192373362
2,디딤돌 파이썬 (구조적 프로그래밍으로 설명한 파이썬 기초),https://search.shopping.naver.com/book/catalog...,이찬수,26100,인피니티북스,20220823,9791192373058


In [15]:
books_df['publisher'].unique()

array(['이지스퍼블리싱', '한빛미디어', '기한재', '생능출판', '렉스미디어닷넷', '북두', '복두출판사',
       '에듀웨이', '인포앤북', '정보문화사', '도서출판 홍릉(홍릉과학출판사)', '다본', '디지털북스',
       '에피스테메', '인피니티북스', '한빛아카데미', '자유아카데미', '길벗', '클라우드북스', '퍼플',
       '한국방송통신대학교출판문화원', '성안당', '북랩', '그린', '탐진출판사', '영진닷컴', '연두에디션',
       '에이콘출판', '시그마프레스', '길벗캠퍼스', '제이펍'], dtype=object)

In [ ]:
def filter_books_by_publisher(df, publisher_name):
    """
    특정 출판사가 포함된 도서만 필터링 (image, description 컬럼 제외)

    Parameters:
        df (DataFrame): 도서 데이터프레임
        publisher_name (str): 포함할 출판사 이름

    Returns:
        DataFrame: 필터링된 결과
    """
    return (
        df.loc[df['publisher'].str.contains(publisher_name), df.columns.drop(['image', 'description'])]
          .reset_index(drop=True)
    )

filter_books_by_publisher(books_df,'인피니티북스')

,title,link,author,discount,publisher,pubdate,isbn
0,혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서),https://search.shopping.naver.com/book/catalog...,윤인성,19800,한빛미디어,20220601,9791162245651
1,밑바닥부터 시작하는 딥러닝 1(리마스터판) (파이썬으로 익히는 딥러닝 이론과 구현),https://search.shopping.naver.com/book/catalog...,사이토 고키,23400,한빛미디어,20250124,9791169213387
2,파이썬 3 (프로그래밍을 배우기에 가장 재미있는 언어),https://search.shopping.naver.com/book/catalog...,박상현,0,한빛미디어,20160210,9788968482359


In [17]:
import pandas as pd

shops_df = pd.read_json('../data/shops.json')
print(type(shops_df))
shops_df.head(2)
shops_df.info()

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        50 non-null     object
 1   link         50 non-null     object
 2   image        50 non-null     object
 3   lprice       50 non-null     int64 
 4   hprice       50 non-null     object
 5   mallName     50 non-null     object
 6   productId    50 non-null     int64 
 7   productType  50 non-null     int64 
 8   brand        50 non-null     object
 9   maker        50 non-null     object
 10  category1    50 non-null     object
 11  category2    50 non-null     object
 12  category3    50 non-null     object
 13  category4    50 non-null     object
dtypes: int64(3), object(11)
memory usage: 5.6+ KB


In [26]:
shops_df.loc[shops_df['lprice'] <= 50000,['brand','lprice','mallName','link']]\
    .sort_values(by='lprice').reset_index(drop=True)

,brand,lprice,mallName,link
0,BNX,13600,네이버,https://search.shopping.naver.com/catalog/5638...
1,GGPX,15790,네이버,https://search.shopping.naver.com/catalog/5726...
2,스파오,15900,네이버,https://search.shopping.naver.com/catalog/5629...
3,BNX,17730,네이버,https://search.shopping.naver.com/catalog/5724...
4,GGPX,17780,네이버,https://search.shopping.naver.com/catalog/5728...
5,로엠,17910,네이버,https://search.shopping.naver.com/catalog/5267...
6,제이플로우,18900,제이플로우,https://smartstore.naver.com/main/products/124...
7,GGPX,19920,네이버,https://search.shopping.naver.com/catalog/5724...
8,,22900,ARUMY,https://smartstore.naver.com/main/products/546...
9,후아유,23650,네이버,https://search.shopping.naver.com/catalog/5602...


In [19]:
def filter_and_sort_shops(df, max_price=50000):
    """
    최대 가격 이하 상품 필터링 후 가격 기준 오름차순 정렬

    Parameters:
        df (DataFrame): 쇼핑 데이터프레임
        max_price (int): 최대 가격 기준 (기본값 50000)

    Returns:
        DataFrame: 필터링 및 정렬된 결과
    """
    return (
        df.loc[df['lprice'] <= max_price, ['brand', 'lprice', 'mallName', 'link']]
          .sort_values(by='lprice')
          .reset_index(drop=True)
    )

filter_and_sort_shops(shops_df,20000)

,brand,lprice,mallName,link
0,BNX,13600,네이버,https://search.shopping.naver.com/catalog/5638...
1,GGPX,15790,네이버,https://search.shopping.naver.com/catalog/5726...
2,스파오,15900,네이버,https://search.shopping.naver.com/catalog/5629...
3,BNX,17730,네이버,https://search.shopping.naver.com/catalog/5724...
4,GGPX,17780,네이버,https://search.shopping.naver.com/catalog/5728...
5,로엠,17910,네이버,https://search.shopping.naver.com/catalog/5267...
6,제이플로우,18900,제이플로우,https://smartstore.naver.com/main/products/124...
7,GGPX,19920,네이버,https://search.shopping.naver.com/catalog/5724...


In [20]:
shops_df.columns

Index(['title', 'link', 'image', 'lprice', 'hprice', 'mallName', 'productId',
       'productType', 'brand', 'maker', 'category1', 'category2', 'category3',
       'category4'],
      dtype='object')

In [21]:
shops_df.loc[shops_df['mallName'] == '네이버','lprice':'brand']\
    .sort_values(by='lprice').reset_index(drop=True)

,lprice,hprice,mallName,productId,productType,brand
0,13600,,네이버,56384375434,1,BNX
1,15790,,네이버,57261197505,1,GGPX
2,15900,,네이버,56294882190,1,스파오
3,17730,,네이버,57245160589,1,BNX
4,17780,,네이버,57285402765,1,GGPX
5,17910,,네이버,52675721484,1,로엠
6,19920,,네이버,57245251058,1,GGPX
7,23650,,네이버,56026192332,1,후아유
8,26190,,네이버,57289484260,1,제너럴아이디어
9,28100,,네이버,57727760539,1,


In [22]:
shops_df['mallName'].unique()

array(['트위티 155', '부스더샵', '네이버', 'ARUMY', '세컨찬스라이프', '레이바쿠', '나우인뉴욕',
       '아임재팬', '션타운', '제이플로우', '엠클로', '미드시티 여성니트', '브랜드리퍼블릭', '브랜드사는이쁜언니'],
      dtype=object)

In [23]:
shops_df['brand'].unique()

array(['폴로랄프로렌', '라코스테', '', '꼼데가르송', '제이플로우', '헤지스', '로엠', '지고트',
       '수아레우먼', '제이슨우', '제너럴아이디어', 'BNX', '후아유', '스튜디오톰보이', '유니클로',
       'GGPX', '스파오', '마인드브릿지', '에디션', '비비안웨스트우드', '쉬즈미스', '조르쥬레쉬'],
      dtype=object)

In [24]:
def filter_shops_by_mall(df, mall_name='네이버'):
    """
    특정 쇼핑몰 상품만 필터링 후 가격 기준 오름차순 정렬

    Parameters:
        df (DataFrame): 쇼핑 데이터프레임
        mall_name (str): 쇼핑몰 이름 (기본값 '네이버')

    Returns:
        DataFrame: 필터링 및 정렬된 결과
    """
    return (
        df.loc[df['mallName'] == mall_name, 'lprice':'brand']
          .sort_values(by='lprice')
          .reset_index(drop=True)
    )

filter_shops_by_mall(shops_df,'나우인뉴욕')

,lprice,hprice,mallName,productId,productType,brand
0,87900,,나우인뉴욕,84216650390,2,폴로랄프로렌


In [25]:
def filter_shops(df, keyword, search_type='mall'):
    """
    mallName 또는 brand 기준으로 필터링 후 가격 오름차순 정렬

    Parameters:
        df (DataFrame): 쇼핑 데이터프레임
        keyword (str): 검색할 값 (예: '네이버', '폴로랄프로렌')
        search_type (str): 'mall' or 'brand' (기본값 'mall')

    Returns:
        DataFrame: 필터링 및 정렬된 결과
    """
    
    if search_type == 'mall':
        condition = df['mallName'] == keyword
    elif search_type == 'brand':
        condition = df['brand'] == keyword
    else:
        raise ValueError("search_type은 'mall' 또는 'brand'만 가능합니다.")

    return (
        df.loc[condition, 'lprice':'brand']
          .sort_values(by='lprice')
          .reset_index(drop=True)
    )

filter_shops(shops_df,'라코스테','brand')

,lprice,hprice,mallName,productId,productType,brand
0,178370,,네이버,58281375602,1,라코스테
1,184670,,네이버,56317700933,1,라코스테
2,190970,,네이버,57450800128,1,라코스테
3,197270,,네이버,57396186468,1,라코스테
4,217760,,네이버,58617913471,1,라코스테
5,219870,,네이버,57452522365,1,라코스테
6,251370,,네이버,57080809004,1,라코스테
